In [148]:
import sqlite3 as db
import pandas as pd
#import featuretools as ft
import json
import random

<center><h2><b>Leer DB</b></h2></center>

In [149]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

/tmp/ipykernel_33348/759616273.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [150]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

In [151]:
# Eliminar filas duplicadas
for df_name in dfs:
    dfs[df_name] = dfs[df_name].drop_duplicates()

<center><h2><b>admissionDx</b></h2></center>

In [152]:
# TODO: ? Posiblemente dropear en virtud de patient.apacheAdmissionDx

dfs['admissionDx']

,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,2900423,162,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
1,2900423,162,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|No,No,No
2,2900423,162,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Sepsis, pulmonary","Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,944,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Cardiovascular|Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,944,admission diagnosis|Non-operative Organ Systems|Organ System|Cardiovascular,Cardiovascular,Cardiovascular
...,...,...,...,...,...
7573,2898513,50,"admission diagnosis|All Diagnosis|Non-operative|Diagnosis|Gastrointestinal|Encephalopathy, hepatic","Encephalopathy, hepatic","Encephalopathy, hepatic"
7574,2899929,11,admission diagnosis|Was the patient admitted from the O.R. or went to the O.R. within 4 hours of admission?|Yes,Yes,Yes
7575,2899929,11,admission diagnosis|Elective|Yes,Yes,Yes
7576,2899929,11,"admission diagnosis|All Diagnosis|Operative|Diagnosis|Genitourinary|TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy","TURP, transurethral prostate resection for benign prostatic hypertrophy"


In [153]:
# Drop columns
dfs['admissionDx'] = dfs['admissionDx'].drop(columns=[
    'admitdxtext', # En la gran mayoría de los casos es igual a admitdxname
], axis=1)

In [154]:
#
# Nota: Es una mierda porque tiene mucho más sentido con onehot, pero hay un huevo de diagnósticos y como 10~20 notas adicionales
# TODO: diag1 = 4 ??
#

# admitdxpath - admitdxname: Cada patientunistayid tiene n filas relacionadas con todos las diagnosis encontradas.
def parse_admitdx(group):
    pusID, group_df = group
    diagnosis = []
    additionalinfo = []
    
    for i, row in group_df.iterrows():
        if row['admitdxname'] in ('No', 'Yes'):
            path_parts = row['admitdxpath'].split('|')
            mode = path_parts[-2]
            
            additionalinfo.append(mode + ' ' + row['admitdxname'])
        else:
            diagnosis.append(row['admitdxname'])
            
    # Rellenar 3 huecos en diagnosis y 2 en additionalinfo
    diagnosis = (sorted(diagnosis) + ['No' for _ in range(3)])[:3]
    additionalinfo = (sorted(additionalinfo) + ['No' for _ in range(3)])[:3]
    
    return [pusID] + diagnosis + additionalinfo

new_df = pd.DataFrame(columns=['patientunitstayid', 'diag1', 'diag2', 'diag3', 'additional1', 'additional2', 'additional3'])
for i,  group in enumerate(dfs['admissionDx'].groupby('patientunitstayid')):
    row = parse_admitdx(group)
    new_df.loc[i] = row

#new_df

<center><h2><b>allergy</b></h2></center>

In [155]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_colwidth', None)
#sql_query('SELECT * FROM allergy GROUP BY drugname, allergytype').head(400)

dfs['allergy']['drugname'].isna().value_counts()

False    1480
True      995
Name: drugname, dtype: int64

In [156]:
dfs['allergy'] = dfs['allergy'].drop(columns=[
    'allergyoffset', # Preguntar
    'specialtytype', # Poca varianza 2453 – 22
    'drughiclseqno'
], axis=1)

<center><h2><b>apacheapsvar</b></h2></center>

In [157]:
# ID ya dropeada

<center><h2><b>apachepatientresult</b></h2></center>

<center><h2><b>Apachepredvar</b></h2></center>

<center><h2><b>careplancareprovider</b></h2></center>

<center><h2><b>Careplaneol</b></h2></center>

<center><h2><b>careplangeneral</b></h2></center>

<center><h2><b>careplangoal</b></h2></center>

<center><h2><b>careplaninfectiousdisease</b></h2></center>

<center><h2><b>customlab</b></h2></center>

<center><h2><b>diagnosis</b></h2></center>

In [158]:
# Redondear offset al múltiplo de 50 más cercano hacia abajo para eliminar duplicados
# TODO: Hacer en todas las tablas
#dfs['diagnosis']['diagnosisoffset'] = dfs['diagnosis'].apply(lambda row: (int(row['diagnosisoffset'])//50) * 50, axis=1)
#dfs['diagnosis'] = dfs['diagnosis'].drop_duplicates()

In [159]:
# MANTENER 3 DIAGNOSIS MÁS IMPORTANTES POR SECUENCIA Y PRIORIDAD
new_df = pd.DataFrame(columns=['patientunitstayid', 'diag1', 'diag2', 'diag3', 'additional1', 'additional2', 'additional3'])

def clean_diagnosisstring(group):
    pusID, group_df = group
    priority = {
        'Primary': [],
        'Major':  [],
        'Other': []
    }
    
    # Obtener las diagnossis de un paciente
    for i, row in group_df.iterrows():
        if row['diagnosisstring'] == row['diagnosisstring'] and row['diagnosispriority'] == row['diagnosispriority']: # not NaN
            diag = row['diagnosisstring'].split('|')[-1]
            priority_lv = row['diagnosispriority']
            
            priority[priority_lv].append(diag)
    
    return [pusID] + (priority['Primary'] + priority['Major'] + priority['Other'] + ['None', 'None', 'None'])[3:]

new_df = pd.DataFrame(columns=['patientunitstayid', 'last1', 'last2', 'last3'])
for i,  group in enumerate(dfs['diagnosis'].groupby('patientunitstayid')):
    row = clean_diagnosisstring(group)
    new_df.loc[i] = row

dfs['diagnosis'] = new_df

ValueError: cannot set a row with mismatched columns

<center><h2><b>hospital</b></h2></center>

<center><h2><b>Infusiondrug</b></h2></center>

<center><h2><b>intakeoutput</b></h2></center>

In [ ]:
dfs['intakeOutput'] = dfs['intakeOutput'].drop(columns=[
    'intaketotal', # Computada en otra variable
    'outputtotal', # Computada en otra variable
    'cellpath', # Redundante en celltext
    'cellvaluetext' # dependiente de cellvaluenumeric
], axis=1)

In [ ]:
#dfs['intakeOutput']

<center><h2><b>lab</b></h2></center>

In [ ]:
# TODO: Mirar relación entre labmeasurenamesystem y labmeasurenameinterface

dfs['lab'] = dfs['lab'].drop(columns=[
    'labtypeid', # Irrelevante
    'labname', # Irrelevante
    'labresulttext', # Redundante con labResult
    'labresultrevisedoffset' # Reundante con labresultoffset
], axis=1)

In [ ]:
#dfs['lab'] 

<center><h2><b>medication</b></h2></center>

In [ ]:
#dfs['medication']

In [ ]:
# TODO: Parsear dosage con drugname

dfs['medication'] = dfs['medication'].drop(columns=[
    'drugorderoffset', # Irrelevante
    'drugivadmixture', # No parece relevante
    'drugordercancelled', # Irrelevante
    'drugname', # Muy incompleta y redundante con drughiclseqno
    'loadingdose', # Mucho NULL
    'prn', # ? Redundante con frequency
    'gtc' # ? No parece que tenga relevancia
], axis=1)

<center><h2><b>microLab</b></h2></center>

In [ ]:
# TODO ?

<center><h2><b>note</b></h2></center>

In [ ]:
dfs['note'] = dfs['note'].drop(columns=[
    'noteenteredoffset', # Irrelevante
    'notetype', # No parece relevante
    'notevalue', # Parece tener información sobre notetext
], axis=1)

<center><h2><b>nurseAssessment</b></h2></center>

In [ ]:
# TODO: ? Parece que hay múltiples assessments de diferentes enfermeras al mismo tiempo con diagnósticos diferentes.
# TODO: Unir celllabel	cellattribute	cellattributevalue
# TODO: Hacer algo con cellattributepath

dfs['nurseAssessment'] = dfs['nurseAssessment'].drop(columns=[
    'nurseassessentryoffset', # Irrelevante
], axis=1)

<center><h2><b>nurseCare</b></h2></center>

In [ ]:
dfs['nurseCare'] = dfs['nurseCare'].drop(columns=[
    'nursecareentryoffset', # Irrelevante
    'celllabel' # cellattribute es más detallada
], axis=1)

<center><h2><b>nurseCharting</b></h2></center>

In [ ]:
# Hacer algo con esta mierda

dfs['nurseCharting'] = dfs['nurseCharting'].drop(columns=[
    'nursingchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>pastHistory</b></h2></center>

In [ ]:
dfs['pastHistory'] = dfs['pastHistory'].drop(columns=[
    'pasthistoryenteredoffset', # Irrelevante
    'pasthistoryvaluetext' # Igual que pasthistoryvalue
], axis=1)

<center><h2><b>patient</b></h2></center>

In [ ]:
#dfs['patient']

In [ ]:
import math

def clean_age(row):
    age = row['age']
    
    if age == '':
        return 50 # TODO: Median
    elif age == '> 89':
        return 100
    elif math.isnan(float(age)):
        return 50 # ""
    else:
        return int(age)

In [ ]:
#Sepsis, pulmonary              -> 121                                                                                                                                                                    121
#Diabetic ketoacidosis          -> 105                                                                                                                                                                 105
#CHF, congestive heart failure  -> 102   

def randomApache():
    num = random.randint(1, 3)
    
    if(num == 1): 
        return 'CHF, congestive heart failure'
    
    elif (num == 2): 
        return 'Diabetic ketoacidosis'
    
    else: return 'Sepsis, pulmonary'
     
def clean_apacheadmissiondx(row):
    apache = row['apacheadmissiondx']
    
    if apache != apache:
        return randomApache() # ""
    else:
        return apache

In [ ]:
# 177.80    151
# 172.70    146
# 167.60    133
# 165.10    127
                                                                                                                                                    
def randomAdmissionheight():
    num = random.randint(1, 4)
    
    if(num == 1): 
        return 177.80
    
    elif (num == 2): 
        return 172.70
    
    elif (num == 3): 
        return 167.60
    
    else: return 165.10
     
def clean_admissionheight(row):
    admissionheight = row['admissionheight']
    
    if math.isnan(float(admissionheight)):
        return randomAdmissionheight() # ""
    else:
        return admissionheight

In [ ]:
# Random
def clean_gender(row):
    gender = row['gender']
    
    if gender != gender:
        if random.randint(1, 2) % 2 == 0: return 'Male'
        else: return 'Famale'
    else:
        return gender

In [ ]:
# Caucasian
def clean_ethnicity(row):
    ethnicity = row['ethnicity']
    
    if ethnicity != ethnicity:
        return 'Caucasian'
    else:
        return ethnicity

In [ ]:
# Emergency Department
def clean_hospitaladmitsource(row):
    hospitalAdmit = row['hospitaladmitsource']
    
    if hospitalAdmit != hospitalAdmit:
        return 'Emergency Department' # ""
    else:
        return hospitalAdmit


In [ ]:
def clean_admissionweight(row, df):
    admissionweight = row['admissionweight']
    
    if math.isnan(float(admissionweight)):
        return  df[df['age']==row['age']]['admissionweight'].median()
    else:
        return admissionweight    

In [ ]:
# TODO: Feature idea - BMI
# TODO: ? Cual es más importante, unitDischargeOffset u hospitalDischargeOffset  

# Drop
dfs['patient'] = dfs['patient'].drop(columns=[
    'wardid', # ? Tiene pinta de que no va a ser muy importante
    'patienthealthsystemstayid', # Irrelevante
    'hospitaldischargeyear', # Irrelevante. Solo hay datos de los años 2014 y 2015
    'hospitaladmittime24', # Drop a priori. No parece importante
    'hospitaldischargetime24', # Drop a priori. No parece importante
    'hospitaldischargelocation', # No parece muy relevante
    'unittype', # Parece que ya está presente en otras partes
    'unitadmittime24', # No parece muy relevante
    'unitadmitsource', # ?
    'unitstaytype', # No parece muy relevante
    'dischargeweight', # No parece que sea muy relevante el peso cuando esté muerto
    'unitdischargetime24', # Irrelevante
    'unitdischargelocation', # Irrelevante
    'hospitaldischargestatus', # Irrelevante
    'unitdischargestatus',
    'uniquepid'
], axis=1)

# Eliminar NaN TODO
#dfs['patient'] = dfs['patient'].dropna()

# Age
dfs['patient']['age']                 = dfs['patient'].apply(clean_age, axis=1)

# Apacheadmissiondx
dfs['patient']['apacheadmissiondx']   = dfs['patient'].apply(clean_apacheadmissiondx, axis=1)

# Hospitaladmitsource
dfs['patient']['hospitaladmitsource'] = dfs['patient'].apply(clean_hospitaladmitsource, axis=1)

# Gender
dfs['patient']['gender']              = dfs['patient'].apply(clean_gender, axis=1)

# ethnicity
dfs['patient']['ethnicity']           = dfs['patient'].apply(clean_ethnicity, axis=1)

# admissionheight
dfs['patient']['admissionheight']     = dfs['patient'].apply(clean_admissionheight, axis=1)

# admissionweight -> OUTLAYERS TODO:
dfs['patient']['admissionweight']     = dfs['patient'].apply(lambda row : clean_admissionweight(row, dfs['patient']), axis=1)

In [ ]:
import sqlite3
import random

# Emergency Department
# print (dfs['patient']['hospitaladmitsource'].value_counts())

print (dfs['patient']['admissionweight'].value_counts())

<center><h2><b>physicalExam</b></h2></center>

In [ ]:
# TODO: Darle una limpieza fuerte

dfs['physicalExam'] = dfs['physicalExam'].drop(columns=[
    'physicalexamoffset', # Irrelevante
], axis=1)

<center><h2><b>respiratoryCare</b></h2></center>

In [ ]:
dfs['respiratoryCare'] = dfs['respiratoryCare'].drop(columns=[
    'airwaysize', 'airwayposition', 'cuffpressure', 'apneaparams', 'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit', 'lowpeakpreslimit',
    'hirespratelimit', 'lowrespratelimit', 'sighpreslimit', 'lowironoxlimit', 'highironoxlimit', 'meanairwaypreslimit', 'peeplimit', 'cpaplimit',
    'setapneainterval', 'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow', 'setapneainsptime', 'setapneaie', 'setapneafio2'
], axis=1)

<center><h2><b>respiratoryCharting</b></h2></center>

In [ ]:
# TODO: ?

dfs['respiratoryCharting'] = dfs['respiratoryCharting'].drop(columns=[
    'respchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>treatment</b></h2></center>

In [ ]:
# TODO: ?

dfs['treatment'] = dfs['treatment'].drop(columns=[
    'activeupondischarge', # Irrelevante    
], axis=1)

<center><h2><b>vitalAperiodic</b></h2></center>

In [ ]:
# TODO

dfs['vitalAperiodic'] = dfs['vitalAperiodic'].drop(columns=[
], axis=1)

<center><h2><b>vitalPeriodic</b></h2></center>

In [ ]:
# TODO

dfs['vitalPeriodic'] = dfs['vitalPeriodic'].drop(columns=[
], axis=1)

---

<center><h2><b>Exportar a CSV</b></h2></center>

In [ ]:
#for df_name in dfs:
#    dfs[df_name].to_csv('../db/csv_clean/' + df_name + '.csv', index=False)

---

<center><h2><b>Exportar a SQL</b></h2></center>

In [ ]:
import os

sqlite_path = "../db/sqlite/eicu_v2_0_1_clean.sqlite3"
os.remove(sqlite_path)
conn = db.connect(sqlite_path)

for df_name in dfs:
    dfs[df_name].to_sql(df_name, conn, index=False)